In [1]:
# Main Libraries
from sympy import symbols, Matrix, simplify, expand, collect, Poly

$$ f(x,y,z) = (x+y)^2 + (y+z)^2 = 1 $$ 

In [3]:
V = Matrix([[1,1,0],[1,2,1],[0,1,1]])
V

Matrix([
[1, 1, 0],
[1, 2, 1],
[0, 1, 1]])

In [4]:
eigen_vect = V.eigenvects()
eigen_vect

[(0,
  1,
  [Matrix([
   [ 1],
   [-1],
   [ 1]])]),
 (1,
  1,
  [Matrix([
   [-1],
   [ 0],
   [ 1]])]),
 (3,
  1,
  [Matrix([
   [1],
   [2],
   [1]])])]

In [5]:
U_C = [eigen_vect[i][2][0].normalized() for i in range(len(eigen_vect))]
U_I = Matrix.hstack(*U_C)
U_I

Matrix([
[ sqrt(3)/3, -sqrt(2)/2, sqrt(6)/6],
[-sqrt(3)/3,          0, sqrt(6)/3],
[ sqrt(3)/3,  sqrt(2)/2, sqrt(6)/6]])

In [6]:
U = U_I.inv()
U

Matrix([
[ sqrt(3)/3, -sqrt(3)/3, sqrt(3)/3],
[-sqrt(2)/2,          0, sqrt(2)/2],
[ sqrt(6)/6,  sqrt(6)/3, sqrt(6)/6]])

In [7]:
V_d =  U*V*U_I
V_d.simplify()
V_d

Matrix([
[0, 0, 0],
[0, 1, 0],
[0, 0, 3]])

In [8]:
x , y , z = symbols('x y z')
x_ , y_ , z_ = symbols("x' y' z'")

In [9]:
f = Matrix([x,y,z]) 
f

Matrix([
[x],
[y],
[z]])

In [10]:
f_ = Matrix([x_,y_,z_]) 
f = U_I*f_
f.simplify()
f

Matrix([
[sqrt(3)*x'/3 - sqrt(2)*y'/2 + sqrt(6)*z'/6],
[              -sqrt(3)*x'/3 + sqrt(6)*z'/3],
[sqrt(3)*x'/3 + sqrt(2)*y'/2 + sqrt(6)*z'/6]])

In [11]:
x_expr =  f[0]
y_expr =  f[1]
z_expr =  f[2]

In [12]:
expr = ((x + y)**2 + (y + z)**2).expand()
expr

x**2 + 2*x*y + 2*y**2 + 2*y*z + z**2

In [13]:
expr = expr.subs({x: x_expr, y: y_expr, z: z_expr})
expr = expr.simplify()
expr

y'**2 + 3*z'**2

$$ f(x^{'},y^{'},z^{'}) = y^{'}{^2} + 3 z^{'}{^2} = 1 $$ 

---

In [16]:
def extract_quadratic_matrix(expr, vars_):
    expr = expand(expr)
    n = len(vars_)
    M = Matrix.zeros(n, n)
    for i in range(n):
        for j in range(n):
            coeff = Poly(expr, *vars_).coeff_monomial(vars_[i]*vars_[j])
            if i == j:
                M[i, j] = coeff
            else:
                M[i, j] = M[j, i] = coeff / 2
    return M

In [17]:
def Transforming(expr , V= None):
    try:
        if V is None:
            V = extract_quadratic_matrix(expr, vars_xyz)
        eigen_vect = V.eigenvects()
        U_C = [eigen_vect[i][2][0].normalized() for i in range(len(eigen_vect))]
        U_I = Matrix.hstack(*U_C)
        U = U_I.inv()
        f = Matrix([x,y,z]) 
        f_ = Matrix([x_,y_,z_]) 
        f = U_I*f_
        f.simplify()
        x_expr =  f[0]
        y_expr =  f[1]
        z_expr =  f[2]
        expr.expand()
        expr = expr.subs({x: x_expr, y: y_expr, z: z_expr})
        expr = expr.simplify()
        return expr
    except Exception as e:
        return f"Could not transform expression due to: {str(e)}"

In [18]:
x , y , z = symbols('x y z')
x_ , y_ , z_ = symbols("x' y' z'")
vars_xyz = [x, y, z]

In [19]:
expr = (x + y)**2 + (y + z)**2
transformed_expr = Transforming(expr)
transformed_expr

y'**2 + 3*z'**2

In [20]:
# Simple and symmetric expressions
exprs_symmetric = [
    (x + y)**2 + (y + z)**2,
    (x + z)**2 + (x - y)**2,
    (x + y + z)**2,
    (x - y)**2 + (y - z)**2,
    (x - y)**2 + (x + z)**2 + (y + z)**2,
]

# Expressions with mixed and cross terms
'''
exprs_cross_terms = [
    3*x**2 + 4*x*y + 2*y**2 + 2*y*z + z**2,
    2*x**2 + 3*y**2 + 4*z**2 + 5*x*y + 6*x*z + 7*y*z,
    x**2 + y**2 + z**2 + 2*x*y + 2*y*z + 2*x*z,
    5*x**2 - 3*x*y + 4*y**2 + 6*x*z - 2*y*z + z**2,
]
'''
# Geometric or classic forms
exprs_geometric = [
    x**2 + y**2 + z**2,
    x**2 + y**2 - z**2,
    x**2 - y**2 + z**2,
    x**2 + 2*y**2 + 3*z**2,
]

# Combine all for full test set
all_exprs = exprs_symmetric + exprs_geometric

In [21]:
# Apply to all expressions and print
for i, expr in enumerate(all_exprs, 1):
    transformed = Transforming(expr)
    print(f"\n=== Expression {i} ===")
    print(f"Original: {expr}")
    print(f"Transformed: {transformed}")


=== Expression 1 ===
Original: (x + y)**2 + (y + z)**2
Transformed: y'**2 + 3*z'**2

=== Expression 2 ===
Original: (x - y)**2 + (x + z)**2
Transformed: y'**2 + 3*z'**2

=== Expression 3 ===
Original: (x + y + z)**2
Transformed: Could not transform expression due to: A Matrix must be square to invert.

=== Expression 4 ===
Original: (x - y)**2 + (y - z)**2
Transformed: y'**2 + 3*z'**2

=== Expression 5 ===
Original: (x - y)**2 + (x + z)**2 + (y + z)**2
Transformed: Could not transform expression due to: A Matrix must be square to invert.

=== Expression 6 ===
Original: x**2 + y**2 + z**2
Transformed: Could not transform expression due to: A Matrix must be square to invert.

=== Expression 7 ===
Original: x**2 + y**2 - z**2
Transformed: Could not transform expression due to: A Matrix must be square to invert.

=== Expression 8 ===
Original: x**2 - y**2 + z**2
Transformed: Could not transform expression due to: A Matrix must be square to invert.

=== Expression 9 ===
Original: x**2 + 2*

In [22]:
expr = 3*x**2 + 4*x*y + 2*y**2 + 2*y*z + z**2
V = extract_quadratic_matrix(expr, vars_xyz)
eigen_vect = V.eigenvects()
U_C = [eigen_vect[i][2][0].normalized() for i in range(len(eigen_vect))]
U_I = Matrix.hstack(*U_C)
U_I

Matrix([
[(-5/2 - 3/((-1/2 - sqrt(3)*I/2)*(3/2 + sqrt(23)*I/2)**(1/3)) + (2 + (-1/2 - sqrt(3)*I/2)*(3/2 + sqrt(23)*I/2)**(1/3) + 2/((-1/2 - sqrt(3)*I/2)*(3/2 + sqrt(23)*I/2)**(1/3)))**2/2 - 3*(-1/2 - sqrt(3)*I/2)*(3/2 + sqrt(23)*I/2)**(1/3)/2)/sqrt(-19*sqrt(2)*cos(atan(sqrt(23)/3)/3)/8 + 19*sqrt(6)*sin(atan(sqrt(23)/3)/3)/8 + 33/4 - 19*sqrt(3)*I*(3/2 + sqrt(23)*I/2)**(1/3)/8 - 15*sqrt(2)*cos(atan(sqrt(23)/3)/3)/(2 - 2*sqrt(3)*I) + 3*(-2 - sqrt(6)*sin(atan(sqrt(23)/3)/3)/2 + sqrt(2)*cos(atan(sqrt(23)/3)/3)/2 - sqrt(6)*I*cos(atan(sqrt(23)/3)/3)/2 - sqrt(2)*I*sin(atan(sqrt(23)/3)/3)/2 + sqrt(2)*I*sin(atan(sqrt(23)/3)/3)/(1/2 - sqrt(3)*I/2) + sqrt(2)*cos(atan(sqrt(23)/3)/3)/(1/2 - sqrt(3)*I/2))**2/((3/2 + sqrt(23)*I/2)**(1/3) + sqrt(3)*I*(3/2 + sqrt(23)*I/2)**(1/3)) + 9*sqrt(2)*cos(atan(sqrt(23)/3)/3)/(2*(3/2 + sqrt(23)*I/2)**(1/3) + 2*sqrt(3)*I*(3/2 + sqrt(23)*I/2)**(1/3)) - 13*sqrt(2)*I*(3/2 + sqrt(23)*I/2)**(1/3)*sin(atan(sqrt(23)/3)/3)/4 + 13*sqrt(2)*cos(atan(sqrt(23)/3)/3)/(2*(3/2 + s